In [14]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from geopy.geocoders import Nominatim

# Inicializar el geocodificador
geolocator = Nominatim(user_agent="geoapiExercises")

# Cargar el archivo con direcciones
df_direcciones = pd.read_excel('clientes_CABA_town.xlsx')

# Hay que normalizar los datos ! osea corregir puntos y agregar la zona del pais en donde se este realizando el analisis!


In [15]:
from opencage.geocoder import OpenCageGeocode
import concurrent.futures
# Configurar el geocodificador con tu API key de OpenCage
geocoder = OpenCageGeocode('d50f19aaa56d4e6b9fcba72bc90ea5fa')

# Cargar el archivo con direcciones
df_direcciones = pd.read_excel('clientes_CABA_v4.xlsx')

# Crear listas para almacenar las coordenadas
latitudes = []
longitudes = []

# Función para obtener las coordenadas de una dirección
def get_coordinates_opencage(direccion):
    try:
        result = geocoder.geocode(direccion)
        if result and len(result):
            return result[0]['geometry']['lat'], result[0]['geometry']['lng']
        else:
            return None, None
    except Exception as e:
        print(f"Error en {direccion}: {e}")
        return None, None

# Ejecutar geocodificación en paralelo con un máximo de 5 hilos
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    coordenadas = list(executor.map(get_coordinates_opencage, df_direcciones['direccion_normalizada']))

# Desempaquetar las coordenadas en latitud y longitud
latitudes, longitudes = zip(*coordenadas)

# Añadir las coordenadas al DataFrame original
df_direcciones['latitud'] = latitudes
df_direcciones['longitud'] = longitudes

# Guardar el archivo con coordenadas
df_direcciones.to_excel('direcciones_con_coordenada.xlsx', index=False)

# Verificar algunos resultados
print(df_direcciones.head())

          Fecha_boleto                          nom  \
0  2015-05-16 00:00:00          Carlos Luis Sonntag   
1  2015-04-23 00:00:00       Luis Patricio Conforti   
2  2015-04-27 00:00:00  Rodolfo Sebastián Maccarone   
3  2015-05-15 00:00:00               Walter José Bo   
4  2015-05-07 00:00:00       María Daiana Roncaglia   

                       dirección  town Provincia   zip nacionalidad  \
0                    Saraza 1178  CABA      CABA   NaN    Argentina   
1  Virrey del Pino 1.502, 6° "E"  CABA      CABA  1426          NaN   
2       Av Rivadavia 4390 13 "G"  CABA      CABA   NaN          NaN   
3  Av. DON PEDRO DE MENDOZA 2719  CABA      CABA  1169          NaN   
4   Gabriela Mistral 2.850, 5° B  CABA      CABA  1419    Argentina   

      e_civil       sexo fecha_nac  \
0      Casado  Masculino       NaN   
1       VIUDO        NaN       NaN   
2         NaN        NaN       NaN   
3  DIVORCIADO  Masculino       NaN   
4      Casada   Femenino       NaN   

             

In [18]:
# Cargar los límites de los barrios de un shapefile
barrios = gpd.read_file("barrios.shp")

# Convertir latitud y longitud en puntos geográficos
geometry = [Point(xy) for xy in zip(df_direcciones['longitud'], df_direcciones['latitud'])]
geo_direcciones = gpd.GeoDataFrame(df_direcciones, geometry=geometry)

# Asegurarse de que las coordenadas están en el sistema de referencia correcto (EPSG:4326)
geo_direcciones.set_crs(epsg=4326, inplace=True)

# Verificar que los barrios tienen el mismo CRS y convertir si es necesario
if barrios.crs != geo_direcciones.crs:
    barrios = barrios.to_crs(geo_direcciones.crs)

# Realizar la operación espacial (point-in-polygon) para asignar cada punto a un barrio
# Cambiar 'op' a 'predicate' como se recomienda en versiones recientes de geopandas
direcciones_con_barrio = gpd.sjoin(geo_direcciones, barrios, how="left", predicate='within')

# Visualizar el resultado, mostrando la dirección y el nombre del barrio al que pertenece
print(direcciones_con_barrio[['dirección', 'nombre']])
direcciones_con_barrio.to_excel('BARRIOS_CABA.xlsx', index=False)


                         dirección            nombre
0                      Saraza 1178  PARQUE CHACABUCO
1    Virrey del Pino 1.502, 6° "E"           PALERMO
2         Av Rivadavia 4390 13 "G"      VILLA CRESPO
3    Av. DON PEDRO DE MENDOZA 2719              BOCA
4     Gabriela Mistral 2.850, 5° B  VILLA PUEYRREDON
..                             ...               ...
516           Caracas 387 Dpto 202            FLORES
517                    Conesa 3157             NUÑEZ
518                    Portela 184            FLORES
519     Tres De Febrero 1.066, 3 A      VILLA CRESPO
520            Palestina 645, 2º A      VILLA CRESPO

[521 rows x 2 columns]


## sigo con GBA


In [19]:
gba_dict = {
    'GBA Norte': ['Vicente López', 'San Isidro', 'San Fernando', 'Tigre', 'San Martín', 'Pilar', 'Belén de Escobar', 
                  'Los Polvorines', 'José C. Paz', 'San Miguel', 'Olivos', 'Beccar', 'Victoria', 'Rincón de Milberg', 
                  'Villa Ballester', 'Del Viso', 'Ingeniero Maschwitz', 'Grand Bourg', 'Bella Vista', 'La Lucila', 
                  'Boulogne', 'Virreyes', 'General Pacheco', 'Villa Lynch', 'Manuel Alberti', 'Matheu', 
                  'Tortuguitas', 'Muñiz', 'Florida', 'Martínez', 'Don Torcuato', 'José León Suárez', 
                  'Presidente Derqui', 'Garín', 'Villa de Mayo', 'Villa Martelli', 'Villa Adelina', 
                  'El Talar', 'Villa Maipú', 'Villa Rosa', 'Maquinista Savio', 'Ing. Pablo Nogués', 
                  'Munro', 'Benavídez', 'Billinghurst', 'Loma Verde', 'Carapachay', 'Ricardo Rojas', 
                  'Loma Hermosa','Ing. Maschwitz','Adolfo Sordeaux'],
    
    'GBA Sur': ['Lomas de Zamora', 'Adrogué', 'Monte Grande', 'Ezeiza', 'Quilmes', 'Berazategui', 
                'Florencio Varela', 'Banfield', 'Burzaco', 'El Jagüel', 'Tristán Suárez', 'Bernal', 
                'Ranelagh', 'Bosques', 'Temperley', 'Claypole', 'Luis Guillón', 'La Unión', 'Ezpeleta', 
                'Hudson', 'Villa Brown', 'Villa Fiorito', 'Glew', '9 de Abril', 'San Francisco Solano', 
                'Villa España', 'Villa San Luis', 'Villa Centenario', 'Longchamps', 'Canning','Gerli',
                'Presidente Perón', 'San José', 'Malvinas Argentinas', 'Guernica', 'Ingeniero Budge', 
                'Ministro Rivadavia', 'Rafael Calzada', 'San Francisco Solano','Villa Domínico','Wilde','Valentin Alsina',
                'Avellaneda','Lanús', 'Alte. Brown','Esteban EcheverrÍa','Sarandi'],
    
    'GBA Oeste': ['San Justo', 'Morón', 'Hurlingham', 'Ituzaingó Norte', 'Merlo', 'Moreno', 
                  'General Rodríguez', 'Marcos Paz', 'Ciudadela', 'Ramos Mejía', 'Castelar', 
                  'William C. Morris', 'Ituzaingó Sur', 'San Antonio de Padua', 'Paso del Rey', 'Caseros', 
                  'Lomas del Mirador', 'Haedo', 'Villa Tesei', 'Villa Udaondo', 'Libertad', 'Trujui', 
                  'Santos Lugares', 'Villa Luzuriaga', 'El Palomar', 'Parque San Martín', 'Cuartel V', 
                  'Villa Bosch', 'Ciudad Evita', 'Francisco Álvarez', 'Martín Coronado', 'Laferrere', 
                  'La Reja', 'José Ingenieros', 'Rafael Castillo', 'Sáenz Peña', 'González Catán', 
                  'Loma Hermosa', 'Virrey del Pino', 'Ciudad Jardín Lomas del Palomar', 'Isidro Casanova', 
                  'Villa Madero', 'Villa Celina', 'Tablada', 'Tapiales', 'Aldo Bonzi','Pablo Podestá','Saenz Peña','Villa Raffo',
                  'Luján','La Matanza','Mariano Acosta'],
    'Provincia de BA' : ['Tandil','La Plata','Lobos','Junin','Coronel Suárez','Sarmiento','City Bell','Coronel Rosales','Chascomus','Buenos Aires','Gonnet'],
    'Provincias' : ['Corrientes','Santa Fe','Dina Huapi','Toay','Máximo Paz','Chubut','Salta Capital','Dorrego Guaymallen','Cordoba','Alvarez'],
    'Francia' : ['Lyon'],
    'Chile' : ['Pudahuel'],
    'USA' : ['New York'],
    'Bolivia' : ['Zona Pacata Alta'],

}
import pandas as pd
from fuzzywuzzy import process
import numpy as np

# Crear una lista con todas las localidades
all_towns = [town for towns in gba_dict.values() for town in towns]

# Función para hacer fuzzy matching

# Función para hacer fuzzy matching con manejo de NaN
def correct_spelling(town):
    if isinstance(town, str):  # Verifica si el valor es un string válido
        match, score = process.extractOne(town, all_towns)  # Encuentra el match más cercano y su puntaje
        if score > 80:  # Si el puntaje de similitud es mayor que 80 (configurable)
            return match
    return np.nan  # Si el valor no es un string o no hay buen match, retorna NaN


# Cargar el archivo con direcciones
df_GBA = pd.read_csv('Clientes_fuera_de_CABA.csv')

def get_gba_category(town):
    for category, towns in gba_dict.items():
        if town in towns:
            return category
    return 'Sin categoría'


df_GBA['corrected_town'] = df_GBA['town'].apply(correct_spelling)
df_GBA['category'] = df_GBA['corrected_town'].apply(get_gba_category)
df_GBA.to_excel('GBA_Cat.xlsx', index=False)

